Install the required packages.


Clone the repo.

In [3]:
import os
# get siamfc-pytorch source
if not os.path.exists('siamfc-pytorch'):
  !git clone https://github.com/huanglianghua/siamfc-pytorch.git

Download the pretrained weights.

In [4]:
pretrained_file_name = 'siamfc_alexnet_e50.pth'
if not os.path.exists(pretrained_file_name):
  !gdown https://drive.google.com/uc?id=1UdxuBQ1qtisoWYFZxLgMFJ9mJtGVw6n4

Download and unzip the DragonBaby sequence from the Visual Tracker Benchmark. 

In [5]:
file_name = 'DragonBaby.zip'
if not os.path.exists(file_name):
  !gdown http://cvlab.hanyang.ac.kr/tracker_benchmark/seq/DragonBaby.zip
if not os.path.exists('DragonBaby'):
  !unzip DragonBaby.zip

zsh:1: command not found: gdown
unzip:  cannot find or open DragonBaby.zip, DragonBaby.zip.zip or DragonBaby.zip.ZIP.


Update the path.


In [2]:
import sys
sys.path.append('siamfc-pytorch/')

Import the necessary libraries.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import time
import cv2
import os
from collections import namedtuple
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import DataLoader
from got10k.trackers import Tracker
from siamfc import ops
from siamfc.backbones import AlexNetV1
from siamfc.heads import SiamFC
from siamfc.losses import BalancedLoss
from siamfc.datasets import Pair
from siamfc.transforms import SiamFCTransforms
from siamfc.siamfc import Net
from siamfc.siamfc import TrackerSiamFC
import glob


*Define* the paths for the images, the starting rectangle and the pretrained weights. Then execute the tracker on the images. 

In [5]:
from IPython.display import display, Javascript, Image
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import torch
import pandas as pd

In [23]:
frameWidth = 640
frameHeight = 480
box_detect = [220,150,240,330]
init_img=0
cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)
cap.set(10,150)
boxes = []


while cap.isOpened():
    success, img = cap.read()
    if success:
      if init_img == 0:
        img_0 = img
        last_img=img
        init_img =1
      pair_img = [img_0,last_img,img]
      net_path = 'siamfc_alexnet_e50.pth'
      tracker = TrackerSiamFC(net_path=net_path)
      boxes, times = tracker.track(pair_img, box_detect, visualize=False)
      print(boxes[1])
      start_point = (int(boxes[1][0]),int(boxes[1][1]))
      print(start_point)
      end_point = (int(boxes[1][0])+int(boxes[1][2]),int(boxes[1][1])+int(boxes[1][3]))
      print(end_point)

      cv2.rectangle(img, start_point, end_point, (255, 0, 0), 2) 
      last_img=img
      cv2.imshow("Result", img)
      if cv2.waitKey(1) & 0xFF == ord('q'):
          break

[221.11853027 142.17019653 240.         330.        ]
(221, 142)
(461, 472)
[221.11853027 139.93310547 240.         330.        ]
(221, 139)
(461, 469)
[227.82980347 144.4072876  240.         330.        ]
(227, 144)
(467, 474)
[200.98474121 151.11853027 240.         330.        ]
(200, 151)
(440, 481)
[197.76237488 174.00286865 234.88192749 322.96264648]
(197, 174)
(431, 496)
[193.4499054  176.15911865 234.88192749 322.96264648]
(193, 176)
(427, 498)
[197.76237488 178.31533813 234.88192749 322.96264648]
(197, 178)
(431, 500)
[195.6061554  176.15911865 234.88192749 322.96264648]
(195, 176)
(429, 498)
[196.51055908 171.25234985 240.         330.        ]
(196, 171)
(436, 501)
[197.76237488 176.15911865 234.88192749 322.96264648]
(197, 176)
(431, 498)
[184.82499695 176.15911865 234.88192749 322.96264648]
(184, 176)
(418, 498)
[156.79399109 186.9402771  234.88192749 322.96264648]
(156, 186)
(390, 508)
[137.38789368 193.40896606 234.88192749 322.96264648]
(137, 193)
(371, 515)
[122.6865844

: 

In [14]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
init_img = 0
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break
    
    if init_img == 0:
      img_0 = js_to_image(js_reply["img"])
      init_img =1
    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])
    pair_img = [img_0,img]
    net_path = 'siamfc_alexnet_e50.pth'
    tracker = TrackerSiamFC(net_path=net_path)
    boxes, times = tracker.track(pair_img, box_detect, visualize=False)


    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    bbox_array = cv2.rectangle(image, (box[0], box[1]), (box[0]+box[2], box[1]+box[3]), (255, 0, 0), 2) 

    # draw bounding box of the hand (if multiple hands, just the first one)
    #bbox_array = cv2.rectangle(bbox_array, (168,11), (190,53) , (255, 0, 0), 2) 



    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

NameError: name 'video_stream' is not defined